In [1]:
import pandas as pd
import pymongo
from pymongo import MongoClient

In [2]:
# Connect to MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['inf2003_db']  # Name of the database

# Read data from Excel file (Dataset.xlsx)
users_df = pd.read_excel('Dataset.xlsx', sheet_name='Users', parse_dates=['date_joined'])
recipes_df = pd.read_excel('Dataset.xlsx', sheet_name='Recipes')
recipe_info_df = pd.read_excel('Dataset.xlsx', sheet_name='Recipe_Info')
recipe_ratings_df = pd.read_excel('Dataset.xlsx', sheet_name='Recipe_Ratings')
cuisines_df = pd.read_excel('Dataset.xlsx', sheet_name='Cuisines')
dietary_df = pd.read_excel('Dataset.xlsx', sheet_name='Dietary')
favorites_df = pd.read_excel('Dataset.xlsx', sheet_name='Favorite')

# Ensure any invalid or missing dates are handled properly
users_df['date_joined'] = pd.to_datetime(users_df['date_joined'], errors='coerce')

# Fill missing dates with a default date
users_df['date_joined'].fillna(pd.Timestamp('2024-01-01'), inplace=True)

# Get rows where 'date_joined' is missing
missing_date_joined_rows = users_df[users_df['date_joined'].isna()]
print(missing_date_joined_rows)

Empty DataFrame
Columns: [user_id, username, email, password_hashed, bio, profile_pic, date_joined, dietary_id]
Index: []


C:\Users\Fatin Farahin\AppData\Local\Temp\ipykernel_21484\609424251.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  users_df['date_joined'].fillna(pd.Timestamp('2024-01-01'), inplace=True)


In [3]:
# Function to insert data into MongoDB dynamically
def insert_data_from_df(collection_name, df):
    """Inserts data from a DataFrame into MongoDB collection."""
    data_dict = df.to_dict(orient='records')  # Convert DataFrame to dict
    db[collection_name].insert_many(data_dict)  # Insert into MongoDB

# Insert data dynamically into corresponding collections
insert_data_from_df('users', users_df)
insert_data_from_df('recipes', recipes_df)
insert_data_from_df('recipe_info', recipe_info_df)
insert_data_from_df('recipe_ratings', recipe_ratings_df)
insert_data_from_df('cuisines', cuisines_df)
insert_data_from_df('dietary', dietary_df)
insert_data_from_df('favorites', favorites_df)

In [4]:
# Check if the data is inserted correctly by querying the collection
users = db['users'].find().limit(5)  # Fetch the first 5 records
for user in users:
    print(user)

{'_id': ObjectId('6749465960578d884be9ebe0'), 'user_id': 1, 'username': 'AliceSmith', 'email': 'AliceSmith@gmail.com', 'password_hashed': 'ef92b778bafe771e89245b89ecbc08a44a4e166c06659911881f383d4473e94f', 'bio': 'Hey, I’m Alice, a self-proclaimed pasta enthusiast! From traditional Italian\ndishes to experimenting with fusion recipes, I love creating and sharing pasta magic in my kitchen. Every meal is an opportunity to indulge and create memories with food!', 'profile_pic': 'pexels-olly-3769999.jpg', 'date_joined': datetime.datetime(2024, 1, 1, 0, 0), 'dietary_id': 1.0}
{'_id': ObjectId('6749465960578d884be9ebe1'), 'user_id': 2, 'username': 'BobJohnson', 'email': 'bobjohnson@hotmail.com', 'password_hashed': 'ef92b778bafe771e89245b89ecbc08a44a4e166c06659911881f383d4473e94f', 'bio': 'BBQ is my thing. Whether it’s a summer cookout or a cozy winter meal, \nyou’ll always find me perfecting the grill.', 'profile_pic': 'BobJohnson.jpg', 'date_joined': datetime.datetime(2024, 1, 1, 0, 0), 'di

In [5]:
# Import necessary libraries
from db_connection import connect_db

# Connect to MongoDB
db = connect_db()
recipes_collection = db['recipes']

print("Connected to MongoDB!")

Connected to MongoDB!


In [6]:
# Define the missing file mappings
missing_file_mapping = {
    "lemon-meringue-cheesecake": "LemonMeringueCake.jpg",
    "peach-muffins": "PeachMuffins.jpg",
    "sour-cream-pear-pie": "SourCreamPearPie.jpg",
    "apple-pear-sauce": "ApplePearSauce.jpg",
    "salmon-with-pomegranate-glaze": "SalmonPomegranate.jpg",
    "banana-chocolate-chip-cookies": "BananaChocoCookies.jpg",
    "lemon-pretzel-salad-with-mango-and-kiwi": "LemonPretzelSalad.jpg",
    "mango-pineapple-chutney": "MangoPineappleChutne.jpg",
    "hawaiian-pineapple-chicken": "HawaiianPineappleChicken.jpg",
    "avocado-toast-with-egg": "AvocadoToast.jpg",
    "vegetarian-tortilla-soup": "VegetarianTortillaSoup.jpg",
    "mexican-baked-fish": "MexicanBakedFish.jpg",
    "apricot-cheesecake": "ApricotCheesecake.jpg",
    "chococonut-chip-cookies": "ChococonutCookies.jpg"
}

print("Missing file mappings loaded.")

Missing file mappings loaded.


In [7]:
# Normalize titles and update image_src
for recipe in recipes_collection.find():
    normalized_title = recipe['title'].lower().replace(" ", "-")
    
    # Update the database if a match is found
    if normalized_title in missing_file_mapping:
        image_src = missing_file_mapping[normalized_title]
        recipes_collection.update_one(
            {"_id": recipe["_id"]},
            {"$set": {"image_src": image_src}}
        )
        print(f"Updated '{recipe['title']}' with image '{image_src}'.")
    else:
        print(f"No image found for '{recipe['title']}'.")

No image found for 'Chocolate Cake'.
No image found for 'Apple Pie'.
No image found for 'Cinnamon Bun'.
No image found for 'Hot chocolate'.
No image found for 'The Ultimate Berry Crumble'.
No image found for 'Chocolate Crossaint'.
No image found for 'Macarons'.
No image found for 'Chocolate Moussesss'.
No image found for 'Treasure Sandwich'.
No image found for 'Popsicles'.
No image found for 'Strawberry Shortcake'.
No image found for 'Hainanese Chicken Rice'.
No image found for 'New York Cheesecake'.
No image found for 'Apple Oatmeal Crisp'.
No image found for 'Chocolate Covered Cherry Cookies'.
No image found for 'Fig and Onion Spread'.
No image found for 'Fig and Honey Jam with Walnuts'.
No image found for 'Watermelon Lemonaded'.
No image found for 'Simple Lemon Cake'.
Updated 'Lemon Meringue Cheesecake' with image 'LemonMeringueCake.jpg'.
Updated 'Peach Muffins' with image 'PeachMuffins.jpg'.
Updated 'Sour Cream Pear Pie' with image 'SourCreamPearPie.jpg'.
Updated 'Apple Pear Sauce'